In [16]:
import pickle
import pandas as pd
import glob2
import plotly.express as px
import plotly.graph_objects as go
from influxdb import InfluxDBClient
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from plotly.subplots import make_subplots
import log_loader_time
from influxdb_client import InfluxDBClient
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt


In [17]:
client = InfluxDBClient(host='localhost', port=8086)
client.get_list_database()
client.switch_database('GUFI_DB')
q_string='SELECT mean("Imonreal_G1") FROM "HVmeas" WHERE ("Layer" = \'2\') AND time > \'2020-08-25T00:00:00Z\' and time < \'2020-10-25T23:00:00Z\'  GROUP BY time(5m)'
result=client.query(q_string,epoch='s')
points=result.get_points()
dict_4_pd={
    "time":[],
    "current":[]
}
for point in points:
    dict_4_pd["time"].append(point["time"]+8*3600)
#     if point["mean"]==None:
#             dict_4_pd["humidity"].append(0)
#     else:
#         dict_4_pd["humidity"].append(point["mean"])
    dict_4_pd["current"].append(point["mean"])
current_pd=pd.DataFrame(dict_4_pd)
current_pd['time']=pd.to_datetime(current_pd['time'],unit='s')
current_pd=current_pd.set_index("time")

Exception ignored in: <function InfluxDBClient.__del__ at 0x7f2b54703d90>
Traceback (most recent call last):
  File "/home/alb/anaconda3/lib/python3.7/site-packages/influxdb_client/client/influxdb_client.py", line 169, in __del__
    if self.api_client:
AttributeError: 'InfluxDBClient' object has no attribute 'api_client'


TypeError: __init__() missing 2 required positional arguments: 'url' and 'token'

NameError: name 'point' is not defined

In [15]:
client = InfluxDBClient(host='localhost', port=8086)
client.get_list_database()
client.switch_database('GUFI_DB')
result=client.query('SELECT mean("Humidity") FROM "offline_data"."Environment" WHERE ("Sensor" = \'Clean_Room_ext_supporto\') AND time > \'2020-08-25T00:00:00Z\' and time < \'2020-10-25T23:00:00Z\' GROUP BY time(5m)',epoch='s')
points=result.get_points()
dict_4_pd={
    "time":[],
    "humidity":[]
}
for point in points:
    dict_4_pd["time"].append(point["time"]+28800)
#     if point["mean"]==None:
#             dict_4_pd["humidity"].append(0)
#     else:
#         dict_4_pd["humidity"].append(point["mean"])
    dict_4_pd["humidity"].append(point["mean"])
humidity_pd=pd.DataFrame(dict_4_pd)
humidity_pd['time']=pd.to_datetime(humidity_pd['time'],unit='s')
humidity_pd=humidity_pd.set_index("time")

In [16]:
savefilename="/media/alb/Removibile/graal_extract/cluster_dataframe"
with open (savefilename,"rb+") as savefile:
    cluster_pd=pickle.load(savefile)  

In [17]:
cluster_pd['time'] = pd.to_datetime(cluster_pd['time'],unit='s')
cluster_pd=cluster_pd.set_index("time")
cluster_pd["hour"]=cluster_pd.index.hour
cluster_pd["day"]=cluster_pd.index.date


In [18]:
cluster_pd["Cluster_charge_L1_X"]=(cluster_pd["Cluster_charge_L1top_spX"]+cluster_pd["Cluster_charge_L1bot_spX"])/2
cluster_pd["Cluster_charge_L2_X"]=(cluster_pd["Cluster_charge_L2top_spX"]+cluster_pd["Cluster_charge_L2bot_spX"])/2
cluster_pd["Cluster_charge_L1_V"]=(cluster_pd["Cluster_charge_L1top_spV"]+cluster_pd["Cluster_charge_L1bot_spV"])/2
cluster_pd["Cluster_charge_L2_V"]=(cluster_pd["Cluster_charge_L2top_spV"]+cluster_pd["Cluster_charge_L2bot_spV"])/2

cluster_pd["Cluster_size_L1_X"]=(cluster_pd["Cluster_size_L1top_spX"]+cluster_pd["Cluster_size_L1bot_spX"])/2
cluster_pd["Cluster_size_L2_X"]=(cluster_pd["Cluster_size_L2top_spX"]+cluster_pd["Cluster_size_L2bot_spX"])/2
cluster_pd["Cluster_size_L1_V"]=(cluster_pd["Cluster_size_L1top_spV"]+cluster_pd["Cluster_size_L1bot_spV"])/2
cluster_pd["Cluster_size_L2_V"]=(cluster_pd["Cluster_size_L2top_spV"]+cluster_pd["Cluster_size_L2bot_spV"])/2


In [19]:
dict_new_col_2={
    "current":[]
}
for indexs, row in cluster_pd.iterrows():
    a=(current_pd.current[current_pd.index.get_loc(indexs, method='nearest')])
    dict_new_col_2["current"].append(a)
cluster_pd.insert(1, "current", dict_new_col_2["current"], True) 

In [20]:
dict_new_col={
    "humidity":[]
}
for indexs, row in cluster_pd.iterrows():
    a=(humidity_pd.humidity[humidity_pd.index.get_loc(indexs, method='nearest')])
    dict_new_col["humidity"].append(a)
cluster_pd.insert(1, "humidity", dict_new_col["humidity"], True) 

In [21]:
def plot_parameter_by_current_run(param,run):
    fig=px.scatter(x=cluster_pd.current[cluster_pd["run"]>run],y=cluster_pd[param][cluster_pd["run"]>run],color=cluster_pd.run.astype('str')[cluster_pd["run"]>run],color_discrete_sequence=px.colors.qualitative.G10,trendline="ols")
    fig.update_xaxes(title="G1 ImonReal [uA]")
    if "charge" in param:
        title= param + " [fC]"
    else:
        title= param + " "
    fig.update_layout(title=dict(text=title + " Vs current", x=0.5))
    fig.update_yaxes(title=title)
    fig.write_html("exp_plots/Current_vs_{}.html".format(param), include_plotlyjs="CDN")
    fig.show()

In [22]:
cluster_pd

,run,humidity,current,subrun,Number_of_cluster_L1top_spX,Number_of_cluster_L1bot_spX,Number_of_cluster_L2top_spX,Number_of_cluster_L2bot_spX,Number_of_cluster_L1top_spV,Number_of_cluster_L1bot_spV,...,hour,day,Cluster_charge_L1_X,Cluster_charge_L2_X,Cluster_charge_L1_V,Cluster_charge_L2_V,Cluster_size_L1_X,Cluster_size_L2_X,Cluster_size_L1_V,Cluster_size_L2_V
time,,,,,,,,,,,,,,,,,,,,,
2020-04-14 17:05:31,396,46.420870,NaN,2,61.0,64.0,58.0,60.0,44.0,56.0,...,17,2020-04-14,83.91435,101.69980,63.08310,91.63595,3.238460,2.833335,2.567575,2.507580
2020-04-14 17:16:29,396,46.420870,NaN,3,79.0,76.0,65.0,71.0,59.0,61.0,...,17,2020-04-14,81.32910,95.57805,63.53310,83.75330,3.225345,2.691355,2.549040,2.305930
2020-04-14 17:27:25,396,46.420870,NaN,4,69.0,68.0,64.0,63.0,50.0,60.0,...,17,2020-04-14,94.28310,102.86945,61.37170,88.48795,3.400860,2.928885,2.548210,2.530020
2020-04-14 17:38:37,396,46.420870,NaN,5,76.0,70.0,69.0,71.0,55.0,65.0,...,17,2020-04-14,86.89465,100.98100,60.92050,90.75875,3.221375,2.854190,2.537430,2.461860
2020-04-14 17:50:00,396,46.420870,NaN,6,75.0,78.0,69.0,68.0,57.0,64.0,...,17,2020-04-14,85.82470,104.58800,66.16775,93.06415,3.344390,2.918750,2.633810,2.587980
2020-04-14 18:01:08,396,46.420870,NaN,7,69.0,65.0,68.0,65.0,51.0,57.0,...,18,2020-04-14,86.96405,95.50140,69.79305,79.24510,3.326550,2.867075,2.885380,2.468215
2020-04-14 18:12:16,396,46.420870,NaN,8,64.0,65.0,61.0,62.0,54.0,53.0,...,18,2020-04-14,88.17460,91.77635,63.88660,85.23230,3.274165,2.909800,2.567830,2.608695
2020-04-14 18:23:26,396,46.420870,NaN,9,59.0,56.0,51.0,57.0,44.0,51.0,...,18,2020-04-14,83.89495,99.42890,56.93040,80.47145,3.098620,2.710940,2.509515,2.228695
2020-04-14 18:34:36,396,46.420870,NaN,10,87.0,82.0,81.0,81.0,64.0,75.0,...,18,2020-04-14,83.70825,107.16450,63.24160,88.00435,3.136400,3.010990,2.616530,2.443885


In [23]:
plot_parameter_by_current_run("Cluster_charge_L1_X",417)
plot_parameter_by_current_run("Cluster_charge_L1_V",417)
plot_parameter_by_current_run("Cluster_charge_L2_X",417)
plot_parameter_by_current_run("Cluster_charge_L2_V",417)
plot_parameter_by_current_run("Cluster_size_L1_X",417)
plot_parameter_by_current_run("Cluster_size_L1_V",417)
plot_parameter_by_current_run("Cluster_size_L2_X",417)
plot_parameter_by_current_run("Cluster_size_L2_V",417)

In [24]:
client = InfluxDBClient(host='localhost', port=8086)
client.get_list_database()
client.switch_database('GUFI_DB')
q_string='SELECT mean("Imonreal_G1") FROM "HVmeas" WHERE ("Layer" = \'2\') AND time > \'2020-08-24T00:00:00Z\' and time < \'2020-10-26T23:00:00Z\'  GROUP BY time(15m)'
result=client.query(q_string,epoch='s')
points=result.get_points()
dict_4_pd={
    "time":[],
    "current":[]
}
for point in points:
    dict_4_pd["time"].append(point["time"]+8*3600)
#     if point["mean"]==None:
#             dict_4_pd["humidity"].append(0)
#     else:
#         dict_4_pd["humidity"].append(point["mean"])
    dict_4_pd["current"].append(point["mean"])
current_pd_60m=pd.DataFrame(dict_4_pd)
current_pd_60m['time']=pd.to_datetime(current_pd_60m['time'],unit='s')
current_pd_60m=current_pd_60m.set_index("time")

In [13]:
derivative=np.diff(current_pd_60m["current"])
derivative=np.append(derivative,derivative[-1])
derivative=derivative*4




In [14]:
derivative[derivative<-0.25]=np.NaN
derivative[derivative>1]=np.NaN

/home/alb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning:

invalid value encountered in less

/home/alb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning:

invalid value encountered in greater



In [15]:
current_pd_60m.insert(1,"derivative",derivative,True)

In [16]:
dict_new_col_3={
    "derivative":[]
}
for indexs, row in cluster_pd.iterrows():
    a=(current_pd_60m.derivative[current_pd_60m.index.get_loc(indexs, method='nearest')])
    dict_new_col_3["derivative"].append(a)
cluster_pd.insert(1, "derivative", dict_new_col_3["derivative"], True)

In [17]:
run=417
fig = make_subplots(specs=[[{"secondary_y": False}]])

fig.add_trace( 
    go.Scatter(x=cluster_pd.index[cluster_pd["run"]>run],y=cluster_pd.current[cluster_pd["run"]>run],
              mode="markers",name="current"
              ),)
fig.update_layout(  yaxis_title='G1 ImonRea  [uA]',
                  xaxis_title='Time')
fig.write_html("exp_plots/current.html", include_plotlyjs="CDN")

fig.show()


In [18]:
run=417
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
            go.Scatter(x=cluster_pd.index[cluster_pd["run"]>run],y=cluster_pd.derivative[cluster_pd["run"]>run],name="derivative",mode="markers")
            )
fig.add_trace( 
    go.Scatter(x=cluster_pd.index[cluster_pd["run"]>run],y=cluster_pd.current[cluster_pd["run"]>run],
              mode="markers",name="current"
              ),
    secondary_y=True)
fig.update_layout(title=dict(text='Current and its derivative', x=0.5),
                   yaxis_title='G1 ImonReal derivative (uA/1h)',
                   yaxis2_title='G1 ImonReal ',
                   xaxis_title='Time')
fig.write_html("exp_plots/Derivative_and_current_corretto.html", include_plotlyjs="CDN")

fig.show()


In [19]:
def plot_parameter_by_derivative_run(param,run):
    fig=px.scatter(x=cluster_pd.derivative[cluster_pd["run"]>run],y=cluster_pd[param][cluster_pd["run"]>run],color=cluster_pd.run.astype('str')[cluster_pd["run"]>run],color_discrete_sequence=px.colors.qualitative.G10, trendline="ols")
    fig.update_xaxes(title="G1 ImonReal derivative (uA/1h)")
    if "charge" in param:
        title= param + " [fC]"
    else:
        title= param + " "
    fig.update_layout(title=dict(text=title + " Vs derivative", x=0.5))
    fig.update_yaxes(title=title)
    fig.write_html("exp_plots/Derivative_vs_{}.html".format(param), include_plotlyjs="CDN")

    fig.show()

In [20]:
plot_parameter_by_derivative_run("Cluster_charge_L1_X",413)
plot_parameter_by_derivative_run("Cluster_charge_L1_V",413)
plot_parameter_by_derivative_run("Cluster_charge_L2_X",413)
plot_parameter_by_derivative_run("Cluster_charge_L2_V",413)
plot_parameter_by_derivative_run("Cluster_size_L1_X",413)
plot_parameter_by_derivative_run("Cluster_size_L1_V",413)
plot_parameter_by_derivative_run("Cluster_size_L2_X",413)
plot_parameter_by_derivative_run("Cluster_size_L2_V",413)

In [21]:
run=413
fig=px.scatter(y=cluster_pd.derivative[cluster_pd["run"]>run],x=cluster_pd.humidity[cluster_pd["run"]>run],color=cluster_pd.run.astype('str')[cluster_pd["run"]>run],color_discrete_sequence=px.colors.qualitative.G10)
fig.update_yaxes(title="G1 ImonReal derivative (uA/1h)")
fig.update_layout(title=dict(text="G1 derivative vs humidity", x=0.5))
fig.update_xaxes(title="Humidity [%]")
fig.write_html("exp_plots/Derivative_vs_humidity.html", include_plotlyjs="CDN")

fig.show()

In [22]:
run=413
fig=px.scatter(y=cluster_pd.current[cluster_pd["run"]>run],x=cluster_pd.humidity[cluster_pd["run"]>run],color=cluster_pd.run.astype('str')[cluster_pd["run"]>run],color_discrete_sequence=px.colors.qualitative.G10)
fig.update_yaxes(title="G1 ImonReal  (uA)")
fig.update_layout(title=dict(text="G1 current vs humidity", x=0.5))
fig.update_xaxes(title="Humidity [%]")
fig.write_html("exp_plots/Current_vs_humidity.html", include_plotlyjs="CDN")

fig.show()

In [23]:
run=413
fig=px.scatter(y=cluster_pd.run[cluster_pd["run"]>run],x=cluster_pd.index[cluster_pd["run"]>run],color=cluster_pd.run.astype('str')[cluster_pd["run"]>run],color_discrete_sequence=px.colors.qualitative.G10)
# fig.update_yaxes(title="G1 ImonReal derivative (uA/1h)")
# fig.update_layout(title=dict(text="G1 derivative vs humidity", x=0.5))
# fig.update_xaxes(title="Humidity [%]")
# fig.write_html("exp_plots/Derivative_vs_humidity.html", include_plotlyjs="CDN")

fig.show()

In [24]:
run=413
fig=px.scatter(y=cluster_pd.humidity[cluster_pd["run"]>run],x=cluster_pd.index[cluster_pd["run"]>run],color=cluster_pd.run.astype('str')[cluster_pd["run"]>run],color_discrete_sequence=px.colors.qualitative.G10)
# fig.update_yaxes(title="G1 ImonReal derivative (uA/1h)")
# fig.update_layout(title=dict(text="G1 derivative vs humidity", x=0.5))
# fig.update_xaxes(title="Humidity [%]")
# fig.write_html("exp_plots/Derivative_vs_humidity.html", include_plotlyjs="CDN")

fig.show()

In [25]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
            go.Scatter(y=cluster_pd.humidity[cluster_pd["run"]>run],x=cluster_pd.index[cluster_pd["run"]>run],
           mode='markers',
           marker_color=cluster_pd["run"][cluster_pd["run"]>run],
            marker=dict(
                colorscale='Viridis', # one of plotly colorscales
                showscale=True
            )
            ))
fig.add_trace( 
    go.Scatter(x=cluster_pd.index[cluster_pd["run"]>run],y=cluster_pd.derivative[cluster_pd["run"]>run],
               mode="markers",
              name="derivative"
              ),
    secondary_y=True)
fig.update_layout(title=dict(text='Derivative and humidity', x=0.5),
                   yaxis_title='Humidty',
                   yaxis2_title='G1 ImonReal derivative [uA/h]',
                   xaxis_title='Time')
fig.write_html("exp_plots/Derivative_and_humidity.html", include_plotlyjs="CDN")

fig.show()


In [26]:
client = InfluxDBClient(host='localhost', port=8086)
client.get_list_database()
client.switch_database('GUFI_DB')
result=client.query('SELECT mean("Pressure") FROM "offline_data"."Environment" WHERE ("Sensor" = \'Clean_Room_ext_supporto\') AND time > \'2020-08-25T00:00:00Z\' and time < \'2020-10-25T23:00:00Z\' GROUP BY time(5m)',epoch='s')
points=result.get_points()
dict_4_pd={
    "time":[],
    "pressure":[]
}
for point in points:
    dict_4_pd["time"].append(point["time"]+28800)
#     if point["mean"]==None:
#             dict_4_pd["pressure"].append(0)
#     else:
#         dict_4_pd["pressure"].append(point["mean"])
    dict_4_pd["pressure"].append(point["mean"])
pressure_pd=pd.DataFrame(dict_4_pd)
pressure_pd['time']=pd.to_datetime(pressure_pd['time'],unit='s')
pressure_pd=pressure_pd.set_index("time")

In [27]:
dict_new_col={
    "pressure":[]
}
for indexs, row in cluster_pd.iterrows():
    a=(pressure_pd.pressure[pressure_pd.index.get_loc(indexs, method='nearest')])
    dict_new_col["pressure"].append(a)
cluster_pd.insert(1, "pressure", dict_new_col["pressure"], True) 

In [28]:
run=413
fig=px.scatter(y=cluster_pd.derivative[cluster_pd["run"]>run],x=cluster_pd.pressure[cluster_pd["run"]>run],color=cluster_pd.run.astype('str')[cluster_pd["run"]>run],color_discrete_sequence=px.colors.qualitative.G10)
fig.update_yaxes(title="G1 ImonReal derivative (uA/1h)")
fig.update_layout(title=dict(text="G1 derivative vs pressure", x=0.5))
fig.update_xaxes(title="Pressure [hPa]")
fig.write_html("exp_plots/Derivative_vs_pressure.html", include_plotlyjs="CDN")

fig.show()